In [17]:
import pandas as pd
import numpy as np

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
!pip install scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [19]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('new1_cyber_train.csv')
# Load your testing dataset
test_df = pd.read_csv('new1_cyber_test.csv')


In [22]:
train_df['label'] = train_df['label'].replace({0: 'bully', 1: 'not-bully'})
test_df['label'] = test_df['label'].replace({0: 'bully', 1: 'not-bully'})

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding for training data
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Sequence and Padding for testing data
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')




## Building and Training Models

## DNN

In [25]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 2s - loss: 0.7469 - accuracy: 0.5175 - 2s/epoch - 90ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7281 - accuracy: 0.5042 - 50ms/epoch - 3ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7017 - accuracy: 0.5225 - 57ms/epoch - 3ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6888 - accuracy: 0.5291 - 44ms/epoch - 2ms/step
Epoch 5/30
19/19 - 0s - loss: 0.7001 - accuracy: 0.4908 - 45ms/epoch - 2ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6994 - accuracy: 0.4992 - 47ms/epoch - 2ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6971 - accuracy: 0.5108 - 55ms/epoch - 3ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6958 - accuracy: 0.5175 - 44ms/epoch - 2ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6860 - accuracy: 0.5358 - 42ms/epoch - 2ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6945 - accuracy: 0.5424 - 47ms/epoch - 2ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6856 - accuracy: 0.5208 - 50ms/epoch - 3ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6879 - accuracy: 0.5441 - 45ms/epoch - 2ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6819 - accuracy:

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 2s - loss: 0.8433 - accuracy: 0.4992 - 2s/epoch - 122ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7228 - accuracy: 0.5008 - 131ms/epoch - 7ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7026 - accuracy: 0.5275 - 158ms/epoch - 8ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6908 - accuracy: 0.5075 - 247ms/epoch - 13ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6956 - accuracy: 0.5258 - 142ms/epoch - 7ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6967 - accuracy: 0.5108 - 138ms/epoch - 7ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6929 - accuracy: 0.5141 - 189ms/epoch - 10ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6986 - accuracy: 0.4692 - 137ms/epoch - 7ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6943 - accuracy: 0.5092 - 94ms/epoch - 5ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6935 - accuracy: 0.5008 - 95ms/epoch - 5ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6978 - accuracy: 0.4942 - 94ms/epoch - 5ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6881 - accuracy: 0.5324 - 89ms/epoch - 5ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6865 -

...

19/19 - 1s - loss: 0.8800 - accuracy: 0.4992 - 1s/epoch - 63ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7302 - accuracy: 0.4892 - 64ms/epoch - 3ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7135 - accuracy: 0.4676 - 75ms/epoch - 4ms/step
Epoch 4/30
19/19 - 0s - loss: 0.6967 - accuracy: 0.5125 - 74ms/epoch - 4ms/step
Epoch 5/30
19/19 - 0s - loss: 0.6945 - accuracy: 0.5008 - 60ms/epoch - 3ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6958 - accuracy: 0.4975 - 74ms/epoch - 4ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6969 - accuracy: 0.4958 - 80ms/epoch - 4ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6966 - accuracy: 0.4958 - 60ms/epoch - 3ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6942 - accuracy: 0.4975 - 69ms/epoch - 4ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6926 - accuracy: 0.4942 - 61ms/epoch - 3ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6954 - accuracy: 0.5058 - 56ms/epoch - 3ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6937 - accuracy: 0.5141 - 56ms/epoch - 3ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6941 - accuracy:

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


19/19 - 1s - loss: 0.7997 - accuracy: 0.4950 - 1s/epoch - 56ms/step
Epoch 2/30
19/19 - 0s - loss: 0.7119 - accuracy: 0.5000 - 57ms/epoch - 3ms/step
Epoch 3/30
19/19 - 0s - loss: 0.7107 - accuracy: 0.4917 - 57ms/epoch - 3ms/step
Epoch 4/30
19/19 - 0s - loss: 0.7015 - accuracy: 0.4950 - 52ms/epoch - 3ms/step
Epoch 5/30
19/19 - 0s - loss: 0.7033 - accuracy: 0.4452 - 68ms/epoch - 4ms/step
Epoch 6/30
19/19 - 0s - loss: 0.6971 - accuracy: 0.4734 - 57ms/epoch - 3ms/step
Epoch 7/30
19/19 - 0s - loss: 0.6953 - accuracy: 0.4967 - 76ms/epoch - 4ms/step
Epoch 8/30
19/19 - 0s - loss: 0.6936 - accuracy: 0.5150 - 58ms/epoch - 3ms/step
Epoch 9/30
19/19 - 0s - loss: 0.6935 - accuracy: 0.4817 - 93ms/epoch - 5ms/step
Epoch 10/30
19/19 - 0s - loss: 0.6936 - accuracy: 0.4850 - 67ms/epoch - 4ms/step
Epoch 11/30
19/19 - 0s - loss: 0.6930 - accuracy: 0.5017 - 52ms/epoch - 3ms/step
Epoch 12/30
19/19 - 0s - loss: 0.6916 - accuracy: 0.5299 - 53ms/epoch - 3ms/step
Epoch 13/30
19/19 - 0s - loss: 0.6916 - accuracy:

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
29/29 - 1s - loss: 0.7259 - accuracy: 0.5255 - 1s/epoch - 51ms/step
Epoch 2/30
29/29 - 0s - loss: 0.6995 - accuracy: 0.4978 - 95ms/epoch - 3ms/step
Epoch 3/30
29/29 - 0s - loss: 0.6952 - accuracy: 0.5089 - 77ms/epoch - 3ms/step
Epoch 4/30
29/29 - 0s - loss: 0.6975 - accuracy: 0.4867 - 110ms/epoch - 4ms/step
Epoch 5/30
29/29 - 0s - loss: 0.6935 - accuracy: 0.4911 - 85ms/epoch - 3ms/step
Epoch 6/30
29/29 - 0s - loss: 0.6937 - accuracy: 0.5100 - 102ms/epoch - 4ms/step
Epoch 7/30
29/29 - 0s - loss: 0.6935 - accuracy: 0.5244 - 91ms/epoch - 3ms/step
Epoch 8/30
29/29 - 0s - loss: 0.6953 - accuracy: 0.4967 - 88ms/epoch - 3ms/step
Epoch 9/30
29/29 - 0s - loss: 0.6906 - accuracy: 0.5155 - 99ms/epoch - 3ms/step
Epoch 10/30
29/29 - 0s - loss: 0.6931 - accuracy: 0.4889 - 83ms/epoch - 3ms/step
Epoch 11/30
29/29 - 0s - loss: 0.6923 - accuracy: 0.5166 - 87ms/epoch - 3ms/step
Epoch 12/30
29/29 - 0s - loss: 0.6905 - accuracy: 0.5200 - 103ms/epoch - 4ms/step
Epoch 13/30
29/29 - 0s - loss: 0.69

In [26]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.734976,10
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.771609,1
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.750528,5
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.767158,2
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.742791,9
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.748309,8
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.729505,11
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.752757,4
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.750524,6
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.725057,13


In [27]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.01, 'optimizer': 'RMSprop'}
Best Accuracy (DNN):  0.771609449981543


In [28]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


8/8 - 0s - 83ms/epoch - 10ms/step
Test Accuracy: 0.7831858407079646


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='bully')
recall_spam = recall_score(y_test, y_pred, pos_label='bully')
f1_spam = f1_score(y_test, y_pred, pos_label='bully')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not-bully')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not-bully')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not-bully')

print("Precision (bully):", precision_spam)
print("Recall (bully):", recall_spam)
print("F1-score (bully):", f1_spam)

print("Precision (not-bully):", precision_non_spam)
print("Recall (not-bully):", recall_non_spam)
print("F1-score (not-bully):", f1_non_spam)

Precision (bully): 0.9210526315789473
Recall (bully): 0.6194690265486725
F1-score (bully): 0.7407407407407406
Precision (not-bully): 0.7133333333333334
Recall (not-bully): 0.9469026548672567
F1-score (not-bully): 0.8136882129277567


## LSTM

In [ ]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer=Adam, learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),  # Increased output_dim for more expressive power
        LSTM(128, return_sequences=True),  # Increased LSTM units for more representational capacity
        Dropout(0.5),
        LSTM(128),  # Increased LSTM units for more representational capacity
        Dropout(0.5),  # Increased dropout rate for regularization
        Dense(64, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(32, activation='relu'),  # Increased number of neurons in Dense layers
        Dropout(0.3),  # Added dropout for regularization
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 14s - loss: 0.6968 - accuracy: 0.5092 - 14s/epoch - 736ms/step
Epoch 2/30
19/19 - 3s - loss: 0.6955 - accuracy: 0.5208 - 3s/epoch - 151ms/step
Epoch 3/30
19/19 - 4s - loss: 0.6930 - accuracy: 0.5042 - 4s/epoch - 186ms/step
Epoch 4/30
19/19 - 3s - loss: 0.6933 - accuracy: 0.5175 - 3s/epoch - 177ms/step
Epoch 5/30
19/19 - 3s - loss: 0.6986 - accuracy: 0.5092 - 3s/epoch - 155ms/step
Epoch 6/30
19/19 - 3s - loss: 0.6922 - accuracy: 0.5008 - 3s/epoch - 149ms/step
Epoch 7/30
19/19 - 4s - loss: 0.6950 - accuracy: 0.4958 - 4s/epoch - 187ms/step
Epoch 8/30
19/19 - 3s - loss: 0.6940 - accuracy: 0.4992 - 3s/epoch - 177ms/step
Epoch 9/30
19/19 - 3s - loss: 0.6931 - accuracy: 0.5108 - 3s/epoch - 160ms/step
Epoch 10/30
19/19 - 3s - loss: 0.6930 - accuracy: 0.5108 - 3s/epoch - 162ms/step
Epoch 11/30
19/19 - 4s - loss: 0.6965 - accuracy: 0.4825 - 4s/epoch - 226ms/step
Epoch 12/30
19/19 - 3s - loss: 0.6930 - accuracy: 0.5108 - 3s/epoch - 161ms/step
Epoch 13/30
19/19 - 3s - loss: 0.69

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6953 - accuracy: 0.4992 - 8s/epoch - 402ms/step
Epoch 2/30
19/19 - 4s - loss: 0.6924 - accuracy: 0.5141 - 4s/epoch - 195ms/step
Epoch 3/30
19/19 - 3s - loss: 0.6975 - accuracy: 0.4626 - 3s/epoch - 146ms/step
Epoch 4/30
19/19 - 3s - loss: 0.6946 - accuracy: 0.5025 - 3s/epoch - 143ms/step
Epoch 5/30
19/19 - 3s - loss: 0.6946 - accuracy: 0.4992 - 3s/epoch - 143ms/step
Epoch 6/30
19/19 - 4s - loss: 0.6948 - accuracy: 0.4925 - 4s/epoch - 210ms/step
Epoch 7/30
19/19 - 3s - loss: 0.6938 - accuracy: 0.4825 - 3s/epoch - 145ms/step
Epoch 8/30
19/19 - 3s - loss: 0.6925 - accuracy: 0.4892 - 3s/epoch - 150ms/step
Epoch 9/30
19/19 - 3s - loss: 0.6908 - accuracy: 0.5208 - 3s/epoch - 152ms/step
Epoch 10/30
19/19 - 4s - loss: 0.6887 - accuracy: 0.4725 - 4s/epoch - 215ms/step
Epoch 11/30
19/19 - 3s - loss: 0.6817 - accuracy: 0.5358 - 3s/epoch - 146ms/step
Epoch 12/30
19/19 - 3s - loss: 0.7330 - accuracy: 0.5441 - 3s/epoch - 148ms/step
Epoch 13/30
19/19 - 3s - loss: 0.6896

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
19/19 - 9s - loss: 0.6928 - accuracy: 0.4917 - 9s/epoch - 476ms/step
Epoch 2/30
19/19 - 3s - loss: 0.6977 - accuracy: 0.4601 - 3s/epoch - 158ms/step
Epoch 3/30
19/19 - 3s - loss: 0.6933 - accuracy: 0.4834 - 3s/epoch - 157ms/step
Epoch 4/30
19/19 - 4s - loss: 0.6930 - accuracy: 0.5266 - 4s/epoch - 209ms/step
Epoch 5/30
19/19 - 3s - loss: 0.6943 - accuracy: 0.5050 - 3s/epoch - 167ms/step
Epoch 6/30
19/19 - 3s - loss: 0.6943 - accuracy: 0.4900 - 3s/epoch - 156ms/step
Epoch 7/30
19/19 - 3s - loss: 0.6933 - accuracy: 0.5050 - 3s/epoch - 150ms/step
Epoch 8/30
19/19 - 4s - loss: 0.6949 - accuracy: 0.4884 - 4s/epoch - 216ms/step
Epoch 9/30
19/19 - 3s - loss: 0.6936 - accuracy: 0.4684 - 3s/epoch - 151ms/step
Epoch 10/30
19/19 - 3s - loss: 0.6938 - accuracy: 0.5100 - 3s/epoch - 154ms/step
Epoch 11/30
19/19 - 3s - loss: 0.6922 - accuracy: 0.5083 - 3s/epoch - 155ms/step
Epoch 12/30
19/19 - 4s - loss: 0.6975 - accuracy: 0.4967 - 4s/epoch - 215ms/step
Epoch 13/30
19/19 - 3s - loss: 0.6936

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 8s - loss: 0.6942 - accuracy: 0.4975 - 8s/epoch - 443ms/step
Epoch 2/30
19/19 - 3s - loss: 0.6938 - accuracy: 0.4925 - 3s/epoch - 143ms/step
Epoch 3/30
19/19 - 3s - loss: 0.6939 - accuracy: 0.4842 - 3s/epoch - 143ms/step
Epoch 4/30
19/19 - 4s - loss: 0.6935 - accuracy: 0.5025 - 4s/epoch - 206ms/step
Epoch 5/30
19/19 - 3s - loss: 0.6950 - accuracy: 0.5025 - 3s/epoch - 148ms/step
Epoch 6/30
19/19 - 3s - loss: 0.6942 - accuracy: 0.4642 - 3s/epoch - 147ms/step
Epoch 7/30
19/19 - 3s - loss: 0.6934 - accuracy: 0.5141 - 3s/epoch - 147ms/step
Epoch 8/30
19/19 - 4s - loss: 0.6927 - accuracy: 0.4992 - 4s/epoch - 203ms/step
Epoch 9/30
19/19 - 3s - loss: 0.6946 - accuracy: 0.4859 - 3s/epoch - 142ms/step
Epoch 10/30
19/19 - 3s - loss: 0.6926 - accuracy: 0.5125 - 3s/epoch - 141ms/step
Epoch 11/30
19/19 - 3s - loss: 0.6904 - accuracy: 0.5108 - 3s/epoch - 144ms/step
Epoch 12/30
19/19 - 4s - loss: 0.6877 - accuracy: 0.4809 - 4s/epoch - 203ms/step
Epoch 13/30
19/19 - 3s - loss: 0.6872

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
19/19 - 10s - loss: 0.6946 - accuracy: 0.4992 - 10s/epoch - 521ms/step
Epoch 2/30
19/19 - 3s - loss: 0.6954 - accuracy: 0.4992 - 3s/epoch - 169ms/step
Epoch 3/30
19/19 - 3s - loss: 0.6947 - accuracy: 0.4626 - 3s/epoch - 154ms/step
Epoch 4/30
19/19 - 3s - loss: 0.6940 - accuracy: 0.4908 - 3s/epoch - 154ms/step
Epoch 5/30
19/19 - 4s - loss: 0.6937 - accuracy: 0.4742 - 4s/epoch - 225ms/step
Epoch 6/30
19/19 - 3s - loss: 0.6936 - accuracy: 0.5158 - 3s/epoch - 160ms/step
Epoch 7/30
19/19 - 3s - loss: 0.6929 - accuracy: 0.5341 - 3s/epoch - 166ms/step
Epoch 8/30
19/19 - 3s - loss: 0.6956 - accuracy: 0.4825 - 3s/epoch - 159ms/step
Epoch 9/30
19/19 - 4s - loss: 0.6936 - accuracy: 0.4875 - 4s/epoch - 222ms/step
Epoch 10/30
19/19 - 3s - loss: 0.6928 - accuracy: 0.5008 - 3s/epoch - 165ms/step
Epoch 11/30
19/19 - 3s - loss: 0.6950 - accuracy: 0.5075 - 3s/epoch - 163ms/step
Epoch 12/30
19/19 - 4s - loss: 0.6937 - accuracy: 0.4859 - 4s/epoch - 195ms/step
Epoch 13/30
19/19 - 4s - loss: 0.69

In [ ]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

In [ ]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

In [ ]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='bully')
recall_spam = recall_score(y_test, y_pred, pos_label='bully')
f1_spam = f1_score(y_test, y_pred, pos_label='bully')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not-bully')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not-bully')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not-bully')

print("Precision (bully):", precision_spam)
print("Recall (bully):", recall_spam)
print("F1-score (bully):", f1_spam)

print("Precision (not-bully):", precision_non_spam)
print("Recall (not-bully):", recall_non_spam)
print("F1-score (not-bully):", f1_non_spam)

## Bi-LSTM

In [ ]:
def create_bi_lstm_model(optimizer=Adam, learning_rate=0.001, lstm_units=64, dropout_rate=0.3):
    bi_lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Bidirectional(LSTM(units=lstm_units, return_sequences=True)),
        Bidirectional(LSTM(units=lstm_units)),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    bi_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return bi_lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
bi_lstm_classifier = KerasClassifier(build_fn=create_bi_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search = GridSearchCV(estimator=bi_lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result = grid_search.fit(X_train_padded, y_train)



In [ ]:
results_df = pd.DataFrame(grid_result.cv_results_)
results_df[['params', 'mean_test_score', 'rank_test_score']]
# Print the best parameters and corresponding accuracy

In [ ]:
print("\n",grid_result)
print("\n")
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)


In [ ]:
best_model = grid_result.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='bully')
recall_spam = recall_score(y_test, y_pred, pos_label='bully')
f1_spam = f1_score(y_test, y_pred, pos_label='bully')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not-bully')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not-bully')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not-bully')

print("Precision (bully):", precision_spam)
print("Recall (bully):", recall_spam)
print("F1-score (bully):", f1_spam)

print("Precision (not-bully):", precision_non_spam)
print("Recall (not-bully):", recall_non_spam)
print("F1-score (not-bully):", f1_non_spam)